In [1]:
import flwr as fl
from config import NUM_CLIENTS, DEVICE
from model import VGG16
from main import simulate_cifar


## Centralize training

In [ ]:
from dataloader import load_cifars
trainloaders, valloaders, testloader = load_cifars(1)

trainloader = trainloaders[0]
valloader = valloaders[0]
net = VGG16().to(DEVICE)

for epoch in range(10):
    net.train_epoch(trainloader, 1)
    loss, accuracy = net.test(valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

loss, accuracy = net.test(testloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

Files already downloaded and verified
Files already downloaded and verified


## Fed avg with parameter initialization

In [4]:
from torch import tensor
import torch
from numpy import array

print(tensor(array(0)).shape)
print(torch.Tensor(array(0)).shape)



torch.Size([])
torch.Size([0])


In [2]:
from model import Net
net = VGG16().to(DEVICE)
params = net.get_parameters()

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

simulate_cifar(strategy, net)

Files already downloaded and verified
Files already downloaded and verified


INFO flwr 2023-06-09 17:36:45,461 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-06-09 17:36:47,002	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-09 17:36:47,603 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:10.10.12.34': 1.0, 'CPU': 20.0, 'memory': 4786640487.0, 'object_store_memory': 2393320243.0}
INFO flwr 2023-06-09 17:36:47,604 | server.py:86 | Initializing global parameters
INFO flwr 2023-06-09 17:36:47,605 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-06-09 17:36:47,605 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-06-09 17:36:47,605 | server.py:101 | FL starting
DEBUG flwr 2023-06-09 17:36:47,606 | server.py:218 | fit_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=82654) [Client 8] fit, config: {}


ERROR flwr 2023-06-09 17:37:07,040 | ray_client_proxy.py:87 | Task was killed due to the node running low on memory.
Memory on the node (IP: 10.10.12.34, ID: efc616ad18ed9b7fc1436ea140a65ab128bcac37f2d9dbae16ef51f0) where the task (task ID: 7fc10dc253cbb71054b56bf8898478e5ed0465d501000000, name=launch_and_fit, pid=82654, memory used=1.24GB) was running was 14.73GB / 15.39GB (0.956622), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: efa7a3e129b58274c47fdffe6c7e742ce8b4b3a1c176d82a83172758) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 10.10.12.34`. To see the logs of the worker, use `ray logs worker-efa7a3e129b58274c47fdffe6c7e742ce8b4b3a1c176d82a83172758*out -ip 10.10.12.34. Top 10 memory users:
PID	MEM(GB)	COMMAND
8425	2.84	/snap/pycharm-professional/336/jbr/bin/java -classpath /snap/pycharm-professional/336/lib/app.jar:/s...
82657	1.24	ray::launch_and_fit
82654	1.24

KeyboardInterrupt: 